In [1]:
import random
import sage.graphs.graph_plot
from sage.graphs.base.boost_graph import *
from sage.plot.point import *
import numpy
import timeit
from time import process_time
import time


In [2]:
#generira seznam opažanj cen za grafe, kjer je st_pov_grafa število povezav grafa.

def seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena):
    sur_podatki = [] #seznam vseh opažanj
    for i in range(0, st_opazanj):
        mer_povezav = []
        for j in range(0, st_pov_grafa):
            mer_povezav.append(numpy.random.random_integers(min_cena, max_cena)) #cene so enakomerno porazdeljene v [min_cena, max_cena]
        sur_podatki.append(mer_povezav)
    return(sur_podatki)

#stevilo_pov_grafa = 2 * st_vozlisc^2 - 2 * st_vozlisc, število povezav v n x n mrežastih grafih

In [3]:
#funkcija ki vektor množi s transponiranim vektorjem

def mnoz(vektor1, vektor2):
    matrika = []
    for i in range(0,len(vektor1)):
        vrstica =[]
        for j in range(0, len(vektor2)):
            vrstica.append(vektor1[i] * vektor2[j])
        matrika.append(vrstica)
    return(matrika)

#izračuna matriko, ki specificira obliko in velikost elipsoidne množice negotovosti
def kov_matrika(vse_utezi):
    avr_pov = [i * 1/len(vse_utezi) for i in [sum(x) for x in zip(*vse_utezi)]] #povprečen vektor cen povezav
    zac_mat = mnoz(numpy.subtract(vse_utezi[0],avr_pov),numpy.subtract(vse_utezi[0],avr_pov))
    for i in range(1, len(vse_utezi)):
        zac_mat = numpy.add(zac_mat, mnoz(numpy.subtract(vse_utezi[i],avr_pov),numpy.subtract(vse_utezi[i],avr_pov))) #seštevanje matrik
    kon_mat = numpy.dot(1/len(vse_utezi), zac_mat) #povprečje vseh matrik
    return(kon_mat)

In [4]:
#grafu priredi vektor 0 in 1, ki vrne 1 če je povezava uporabljena na poti med začetnim in končnim vozliščem, in 0 če ni

def vektorcki_x(graf, zacetno, koncno):
    vse_poti = graf.all_paths(zacetno, koncno, use_multiedges=True, report_edges=False, labels=False)
    vse_povezave = graf.edges()
    par = []
    for i in range(0,len(vse_povezave)):
        par.append(vse_povezave[i][0:2])
    seznam_vektorjev = []
    for i in range(0, len(vse_poti)):
        pot = (vse_poti[i])
        prvi_oklepaj1 = list(zip(pot, pot[1:] + pot[:1]))
        prvi_oklepaj = prvi_oklepaj1[:-1]
        x=[]
        for i in range(0, len(par)):
            if par[i] in prvi_oklepaj:
                x.append(1)
            else:
                x.append(0)
        seznam_vektorjev.append(x)
    return(seznam_vektorjev)

In [5]:
#naredi mrežaste grafe, v nadaljevanju je nisva uporabila zaradi njene pomnilniške zahtevnosti

def grid_graf(st_opazanj, st_vozlisc, min_cena, max_cena):
    st_pov_grafa = 2 * st_vozlisc^2 - 2 * st_vozlisc
    utezi = seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena)
    graf_seznam = graphs.Grid2dGraph(st_vozlisc, st_vozlisc).edges()
    nov = [] #najprej spremeniva v list
    for i in range(0,len(graf_seznam)):
        nov.append(list(graf_seznam[i]))
    konec = [] #dodava še ceno, v obliki ki jo sprjema digraph
    seznam_grafov = []
    for j in range(0,len(utezi)):
        zadnji = []
        for i in range(0,len(nov)):
            vmesni = []
            vmesni.append(nov[i][0])
            vmesni.append(nov[i][1])
            vmesni.append(utezi[j][i])
            zadnji.append(tuple(vmesni))
        konec.append(zadnji)
        gr = DiGraph(zadnji) #dan seznam povezav spremeni v usmerjen graf
        seznam_grafov.append(gr)
    return(seznam_grafov, utezi)


In [5]:
#naredi en mrežast graf, to sva v nadaljevanju uporabljala

def min_grid_graf(st_opazanj, st_vozlisc, min_cena, max_cena):
    st_pov_grafa = 2 * st_vozlisc^2 - 2 * st_vozlisc
    utezi = seznam_cen(st_opazanj, st_pov_grafa, min_cena, max_cena)
    graf_oblika = DiGraph(graphs.Grid2dGraph(st_vozlisc, st_vozlisc).edges())
    return(graf_oblika, utezi)


In [6]:
#poišče najkrajšo pot x. 
#Poti je množica iz x, av_pov so povprečne cene povezav, diagonala je diagonala matrike epsilon
#x = argmin{x \in X} <(povprecen c + povprecen d), x>

def argmin(poti, av_pov, diagonala):
    x_min = poti[0]
    cena_pov = numpy.dot(av_pov, x_min)
    cena_d = numpy.dot(diagonala, x_min)
    cena = cena_pov + cena_d
    for i in range(1, len(poti)):
        x_nov = poti[i]
        cena_pov_nov = numpy.dot(av_pov, x_nov)
        cena_d_nov = numpy.dot(diagonala, x_nov)
        cena_nov = cena_pov_nov + cena_d_nov
        if cena_nov < cena:
            x_min = x_nov
            cena_pov = cena_pov_nov
            cena_d = cena_d_nov
            cena = cena_nov
    return(x_min, cena_pov, cena_d, cena)

In [7]:
#poišče leksikografsko najcenejšo rešitev x iz seznama x
#x = arglexmin{x \in X}(<povprecen c, x>, <povprecen d, x>)

def arglexmin(seznam_x, vektorji):
    xmin = min(([numpy.dot(numpy.array(v), numpy.array(x)) for v in vektorji], x) for x in seznam_x)
    return xmin #vrne ceno po prvem vektorju v seznamu in ceno po drugem vektorju v seznamu, ter najcenejšo leksikografsko rešitev

In [0]:
#Kar rabi funkcija razisci2 za poganjanje.

[grafi, weights] = min_grid_graf(50, 9, 1, 100)
diagonalci = numpy.diag(kov_matrika(weights))
povprecje_w = [i * 1/len(weights) for i in [sum(x) for x in zip(*weights)]]
xi = vektorcki_x(grafi, (0, 0), (8, 8))
xl = arglexmin(xi, [povprecje_w, diagonalci]) 
xln = xl[1] #leva leksikografska rešitev
xr = arglexmin(xi, [diagonalci, povprecje_w])
xrn = xr[1] #desna leksikografska rešitev

In [8]:
#slaba praksa algoritma, vendar drugače nama ni šlo, semznam in stevac morata biti pred vsakim poganjanjem funkcije napisana pred njo
#To je naivni algoritem.
#x0 je 

#semznam = []
#stevac = 0
def razisci2(x0, x1, pov, diag, poti):
    global stevac
    alfa_m = numpy.dot(diag,numpy.subtract(x1,x0)) / numpy.dot(numpy.subtract(pov,diag),numpy.subtract(x0,x1))
    v_alfa = numpy.dot(alfa_m, pov)
    v_1alfa = numpy.dot((1-alfa_m), diag)
    vek_zv = numpy.add(v_alfa, v_1alfa)
    x_zv = argmin(poti, v_alfa, v_1alfa)
    stevac = stevac + 1
    stara_najboljsa = argmin([x0], numpy.dot(alfa_m, pov), numpy.dot((1-alfa_m), diag))[3]
    if x_zv[3] < stara_najboljsa:
        #print("Dodali smo novo uč. eks. reš.")
        semznam.append(x_zv[0])
        if -x_zv[3] + stara_najboljsa > 0.01:
            #print("Razlika dovolj velika")
            razisci2(x0, x_zv[0], pov, diag, poti)
            razisci2(x_zv[0], x1, pov, diag, poti)

In [0]:
#Zagon razisci2 oz. naivnega algoritma

semznam = []
stevac = 0
razisci2(xln, xrn, povprecje_w, diagonalci, xi)
print(semznam)
print(stevac)

In [1]:
#zapisuje rezultate, ki jih pridobimo z naivnim algoritmom, na grafih različnih velikosti
#najprej sva definirala različne velikosti, na katerih sva algoritem izvedla, ko sva pognala za grafe velikosti 10x10 pa program ni več zmogel.

import json

tipi_grafov = [3, 4, 5, 6, 7, 8, 9] #velikosti n x n mrežastih grafov
casi = [] #povprecni casi komputacij za grafe različnih velikosti
stetja = [] #povprecno število reševanj optimizacijskega problema najkrajših poti
for i in range(0, len(tipi_grafov)):
    print("TIP GRAFA ", tipi_grafov[i])
    vmesni_casi = []
    vmesna_stetja = []
    for j in range(0, 30):
        [grafi, weights] = min_grid_graf(50, tipi_grafov[i], 1, 100)
        povprecje_w = [i * 1/len(weights) for i in [sum(x) for x in zip(*weights)]]
        diagonalci = numpy.diag(kov_matrika(weights))
        xi = vektorcki_x(grafi, (0, 0), (tipi_grafov[i] - 1, tipi_grafov[i] - 1))
        xl = arglexmin(xi, [povprecje_w, diagonalci])
        xln = xl[1]
        xr = arglexmin(xi, [diagonalci, povprecje_w])
        xrn = xr[1]
        while xln == xrn:  #če sta leksikografski rešitvi slučajno enaki, algoritem ne vrne smiselnih rezultatov, ponovimo
            [grafi, weights] = min_grid_graf(50, tipi_grafov[i], 1, 100)
            povprecje_w = [i * 1/len(weights) for i in [sum(x) for x in zip(*weights)]]
            diagonalci = numpy.diag(kov_matrika(weights))
            xi = vektorcki_x(grafi, (0, 0), (tipi_grafov[i] - 1, tipi_grafov[i] - 1))
            xl = arglexmin(xi, [povprecje_w, diagonalci])
            xln = xl[1]
        xr = arglexmin(xi, [diagonalci, povprecje_w])
        xrn = xr[1]
        semznam = []
        stevac = 0
        start_time = time.time() 
        razisci2(xln, xrn, povprecje_w, diagonalci, xi)
        celoten_cas = (time.time() - start_time) #čas potreben za delovanje naivnega algoritma (funkcija razisci2)
        print("NAREJEN GRAF")
        print("Cas za en graf je", celoten_cas)
        print("Stetje za en graf je", stevac)
        vmesni_casi.append(celoten_cas)
        vmesna_stetja.append(stevac)
    print("Povprecje vmesnih casov je", mean(vmesni_casi))
    casi.append(mean(vmesni_casi))
    print("Povprecje vmesnih stetji je", mean(vmesna_stetja))
    stetja.append(mean(vmesna_stetja))
    print("USTVARJENI PODATKI ZA TIP")
    
with open("rezultati_naiven_na_30.json", 'w') as f:
    f.write(str(casi))
    f.write(str(stetja))